# 3. 의사결정요소 Feature engineering

## 3. 의사결정요소 2 : 접근성
### 교통량
- 추정교통량 '상세도로망_링크ID'와 
- 승용차, 버스, 화물차의 교통량을 모두 더한 '전체' 컬럼만 사용

### 주차장 접근성
- 격자 중심점으로부터 가장 가까운 주차장까지 도로 이동거리
- 국토정보플랫폼 국토정보맵 국토통계지도(http://map.ngii.go.kr/ms/map/NlipMap.do)의 격자별 주차장접근성 데이터(500X500) 사용
- feature_gdf의 250X250 격자를 기준으로 가공 

### 주차장 흡인력
- 허프 확률 모형을 이용하여 주차장이 사람을 유인하는 힘인 주차장 흡인력을 계산함
- 주차장 격자로부터 거리가 가까우면 흡인력이 강함
- 주차장 면적이 넓을수록 흡인력이 강함
- 광양시에 있는 모든 주차장은 모든 격자의 영향을 미칠수 있다고 가정

### 라이브러리 및 데이터 불러오기

In [1]:
from geoband.API import *

In [2]:
import folium
import json
import geopandas as gpd

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import haversine

from shapely.geometry import Point, Polygon, MultiPolygon

import platform

from IPython.display import set_matplotlib_formats

if platform.system() == 'Darwin':
    plt.rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family = 'Malgun Gothic')

plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

import warnings
warnings.filterwarnings(action='ignore')

import pathlib
from tqdm import tqdm, notebook
import os

%matplotlib inline

/opt/app-root/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# 데이터 불러오기
input_path = pathlib.Path('./data')
if not input_path.is_dir():
    input_path.mkdir()

GetCompasData('SBJ_2009_001', '2',  input_path.joinpath('02.광양시_주차장_공간정보.csv'))
GetCompasData('SBJ_2009_001', '10', input_path.joinpath('10.광양시_상세도로망_LV6.geojson'))
GetCompasData('SBJ_2009_001', '11', input_path.joinpath('11.광양시_평일_일별_시간대별__추정교통량_LV6.csv'))

# 국토정보맵 데이터
주차장접근성 = gpd.read_file('./data/주차장접근성/주차장접근성.shp', encoding = 'euc-kr')

# COMPAS 데이터
주차장 = pd.read_csv('./data/02.광양시_주차장_공간정보.csv')
상세도로망 = gpd.read_file('./data/10.광양시_상세도로망_LV6.geojson')
추정교통량 = pd.read_csv('./data/11.광양시_평일_일별_시간대별__추정교통량_LV6.csv')

# feature_gdf
feature_gdf = gpd.read_file('./data/feature_gdf.geojson')

### 교통량

In [4]:
# 추정교통량 '시간적범위'의 데이터 타입을 string으로 통일
추정교통량['시간적범위'] = 추정교통량['시간적범위'].astype(str)

# 시간적_범위 '전일' 추출 및 필요 컬럼만 추출
추정교통량 = 추정교통량[추정교통량['시간적범위'] == '전일']
추정교통량 = 추정교통량[['상세도로망_링크ID','전체']]

# 상세도로망_링크ID 가공(상세도로망 'link_id'과 매칭시키기 위해  상행/하행을 의미하는 두 개 숫자 제거)
추정교통량['상세도로망_링크ID'] = 추정교통량['상세도로망_링크ID'].astype(str)
추정교통량['link_id'] = 추정교통량['상세도로망_링크ID'].str.slice(start = 0, stop = 9)
추정교통량.drop('상세도로망_링크ID', axis = 1, inplace = True)

추정교통량 = 추정교통량.groupby(['link_id'])['전체'].sum().to_frame()
추정교통량.reset_index(level = ['link_id'], inplace = True)

# 상세도로망과 추정교통량 병합
교통량 = pd.merge(상세도로망, 추정교통량, on = 'link_id', how = 'left')
교통량 = 교통량[['link_id', '전체', 'geometry']]
교통량.rename(columns = {'전체':'교통량'}, inplace = True)

In [5]:
# 도로가 지나가는 격자에 교통량 데이터 입력
feature_gdf['교통량'] = 0
for i in tqdm(교통량.index):
    for j in feature_gdf.index:
        if 교통량['geometry'][i].intersects(feature_gdf['geometry'][j]):
            feature_gdf['교통량'][j] += 교통량['교통량'][i]

100%|██████████| 3337/3337 [22:05<00:00,  2.52it/s]


### 주차장 접근성

In [6]:
# 주차장접근성 데이터에서 광양시 데이터만 추출
주차장접근성_광양 = 주차장접근성[주차장접근성['sgg_nm_e'] == 'Gwangyang-si']
주차장접근성_광양.reset_index(inplace = True)

# 주차장접근성_광양 좌표계 변환
주차장접근성_광양 = 주차장접근성_광양.to_crs({'init': 'epsg:4326'})

In [7]:
#주차장접근성 float로 추가
feature_gdf['주차장접근성'] = 0.0
for i in tqdm(주차장접근성_광양.index):
    for j in feature_gdf.index:
        if 주차장접근성_광양['geometry'][i].intersects(feature_gdf['geometry'][j]):
            feature_gdf['주차장접근성'][j] += 주차장접근성_광양['value'][i]

100%|██████████| 1930/1930 [12:11<00:00,  2.64it/s]


In [8]:
# 주차장접근성의 값이 클수록 접근성이 좋지 않으므로, 음수로 취함.
feature_gdf['주차장접근성'] = -(feature_gdf['주차장접근성'])

### 주차장 흡인력

In [9]:
주차장.isnull().sum()

주차장명칭     0
구분        0
면적       29
주차면수      2
유료/무료     0
주소        0
lon       0
lat       0
dtype: int64

- 주차장 면적보다 상대적으로 null값이 적은 주차면수를 활용

In [10]:
# 주차장 흡인력을 계산하는 함수
def parking_power(geometry):
    lat, lon = geometry.centroid.coords[0][1], geometry.centroid.coords[0][0]
    
    total_주차면수 = 0
    for i in range(M):
        주차면수 = 주차장.loc[i, '주차면수']
        y, x = 주차장.loc[i,['lon','lat']].values
        거리 = haversine.haversine((x,y),(lat,lon))

        if not(주차면수 != 주차면수):
            total_주차면수 += (주차면수/(거리))

    return total_주차면수

In [11]:
N = len(feature_gdf)
M = len(주차장)

for i in tqdm(range(N)):
    geometry = feature_gdf.loc[i, 'geometry']
    feature_gdf.loc[i, '주차장흡인력'] = parking_power(geometry)

100%|██████████| 7804/7804 [27:25<00:00,  4.74it/s]


In [12]:
# feature_gdf 저장
feature_gdf.to_file('./data/feature_gdf.geojson', driver = 'GeoJSON')